In [1]:
import numpy as np
import cv2
import os
import sys
from numpy import linalg as LA
from PIL import Image

In [2]:
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml") #人臉辨識器

#Eigen=cv2.face.EigenFaceRecognizer_create()
#Eigen = cv2.face.FisherFaceRecognizer_create()
Eigen = cv2.face.LBPHFaceRecognizer_create()

In [3]:
def loadimage(n):
    x,y=[],[] 
    n=open(n)
    line=n.readlines()
    
    for i in range(len(line)):
        filename,labs=line[i].split(" ") #用空白分開
        x.append(process(filename))#在一個一個放到陣列裡面 回傳陣列方法
        y.append(int(labs))
        
    return x,y

In [13]:
def process(filename):
    image=cv2.imread(filename)
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  #轉成灰階加快檢測速度
    faces=faceCascade.detectMultiScale(image,scaleFactor=1.8,minNeighbors=5)
    #把圖片設定成只有人臉 回傳圖片回去
    #scaleFactor 每次搜尋時減少的倍率,minNeighbors目標至少被檢測五次以上，才認為人臉存在
    #minSize 設定數據搜尋的最小尺寸
    
    if len(faces)>0:
        faces=faces[0] 
        image=cv2.resize(image[faces[0]:faces[0]+faces[2],faces[1]:faces[1]+faces[3]],(250,250))
        
    return image


In [14]:
x,y=loadimage("train.txt")
valx,valy=loadimage("val.txt")

In [15]:
Eigen.train(x,np.array(y)) #np.array 把它變成陣列

In [16]:
Eigen.save("train.xml") #可以暫存檔 重跑方便

In [17]:
from sklearn.metrics import classification_report #回報準確度

a = []

for i in range(0, len(valx)):
    (p,c)=Eigen.predict(valx[i])
    a.append(p)

In [18]:
print (classification_report(valy,a))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          14       0.25      1.00      0.40         1
          15       1.00      1.00      1.00         1
          16       0.00      0.00      0.00         1
          17       0.00    

C:\Users\ioiou\Anaconda3\envs\new\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
import pandas as pd #讀檔
d = pd.read_csv("submission_example.csv")
ID=d["Id"] #圖片名
testimg=[]
testa=[]
testb=[]


for i in range(len(ID)):
    testimg.append(process(ID[i]))

In [20]:
for i in range(0,len(testimg)): #0~100
    (p,c)=Eigen.predict(testimg[i]) #要上傳的檔案
    testa.append(p)
    
    d['Category'].values[i]=p #Category欄位名

In [21]:
d.to_csv('123abcguogadee.csv',index=False) #另存新檔
print (testa)

[1, 76, 52, 94, 62, 2, 63, 68, 6, 86, 51, 73, 73, 88, 94, 73, 93, 69, 5, 54, 85, 77, 20, 73, 16, 59, 84, 9, 51, 41, 42, 85, 85, 29, 6, 75, 37, 9, 30, 74, 76, 23, 73, 95, 16, 93, 40, 46, 19, 94, 75, 79, 20, 48, 53, 82, 18, 25, 7, 12, 19, 17, 20, 12, 84, 74, 63, 33, 82, 62, 19, 55, 42, 21, 76, 57, 90, 23, 73, 20, 34, 80, 67, 52, 69, 99, 37, 36, 94, 81, 16, 63, 77, 10, 77, 11, 61, 44, 20, 95]
